Packages that are being used in this notebook:

In [1]:
from importlib.metadata import version


print("torch version:", version("torch"))
print("tiktoken version:", version("tiktoken"))

torch version: 2.5.1
tiktoken version: 0.9.0


- This notebook provides a brief overview of the data preparation and sampling procedures to get input data "ready" for an LLM
- Understanding what the input data looks like is a great first step towards understanding how LLMs work

<img src="./figures/01.png" width="1000px">

<br>
<br>
<br>
<br>

# 2.1 Tokenizing text

- In this section, we tokenize text, which means breaking text into smaller units, such as individual words and punctuation characters

<img src="figures/02.png" width="800px">

- Load raw text we want to work with
- [The Verdict by Edith Wharton](https://en.wikisource.org/wiki/The_Verdict) is a public domain short story

In [3]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()
    
print("Total number of character:", len(raw_text))
print(raw_text[:120])

Total number of character: 20479
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no great surprise to me 


- The goal is to tokenize and embed this text for an LLM
- Let's develop a simple tokenizer based on some simple sample text that we can then later apply to the text above

<img src="figures/03.png" width="690px">

- The following regular expression will split on whitespaces and punctuation

In [4]:
import re

preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', raw_text)
print("Number of tokens:", len(preprocessed))
print(preprocessed[:38])

Number of tokens: 9235
['I', ' ', 'HAD', ' ', 'always', ' ', 'thought', ' ', 'Jack', ' ', 'Gisburn', ' ', 'rather', ' ', 'a', ' ', 'cheap', ' ', 'genius', '--', 'though', ' ', 'a', ' ', 'good', ' ', 'fellow', ' ', 'enough', '--', 'so', ' ', 'it', ' ', 'was', ' ', 'no', ' ']


In [7]:
unique_tokens = set(preprocessed)
print("Number of unique tokens:", len(unique_tokens))
print(sorted(list(unique_tokens)))

Number of unique tokens: 1133
['', '\n', ' ', '!', '"', "'", '(', ')', ',', '--', '.', ':', ';', '?', 'A', 'Ah', 'Among', 'And', 'Are', 'Arrt', 'As', 'At', 'Be', 'Begin', 'Burlington', 'But', 'By', 'Carlo', 'Chicago', 'Claude', 'Come', 'Croft', 'Destroyed', 'Devonshire', 'Don', 'Dubarry', 'Emperors', 'Florence', 'For', 'Gallery', 'Gideon', 'Gisburn', 'Gisburns', 'Grafton', 'Greek', 'Grindle', 'Grindles', 'HAD', 'Had', 'Hang', 'Has', 'He', 'Her', 'Hermia', 'His', 'How', 'I', 'If', 'In', 'It', 'Jack', 'Jove', 'Just', 'Lord', 'Made', 'Miss', 'Money', 'Monte', 'Moon-dancers', 'Mr', 'Mrs', 'My', 'Never', 'No', 'Now', 'Nutley', 'Of', 'Oh', 'On', 'Once', 'Only', 'Or', 'Perhaps', 'Poor', 'Professional', 'Renaissance', 'Rickham', 'Riviera', 'Rome', 'Russian', 'Sevres', 'She', 'Stroud', 'Strouds', 'Suddenly', 'That', 'The', 'Then', 'There', 'They', 'This', 'Those', 'Though', 'Thwing', 'Thwings', 'To', 'Usually', 'Venetian', 'Victor', 'Was', 'We', 'Well', 'What', 'When', 'Why', 'Yes', 'You', '_',

<br>
<br>
<br>
<br>

# 2.2 Converting tokens into token IDs

- Next, we convert the text tokens into token IDs that we can process via embedding layers later
- For this we first need to build a vocabulary

<img src="figures/04.png" width="900px">

- The vocabulary contains the unique words in the input text

In [8]:
vocab_size = len(unique_tokens)
print(vocab_size)

1133


In [18]:
vocab = {token:integer for integer,token in enumerate(unique_tokens)}
print(vocab)

{'': 0, 'cheeks': 1, 'romantic': 2, 'the': 3, 'come': 4, 'near': 5, 'often': 6, 'lean': 7, 'azaleas': 8, 'leathery': 9, 'mourn': 10, 'idle': 11, 'three': 12, 'most': 13, 'without': 14, 'just': 15, 'none': 16, 'reflection': 17, 'brings': 18, 'profusion': 19, 'fond': 20, 'equally': 21, 'fitting': 22, 'Only': 23, 'disdain': 24, 'wondered': 25, 'faith': 26, 'check': 27, 'Stroud': 28, 'patient': 29, 'underlay': 30, 'himself': 31, 'strolled': 32, 'pardonable': 33, 'might': 34, 'Lord': 35, 'reproduction': 36, 'seemed': 37, 'secret': 38, 'florid': 39, 'moment': 40, 'apparently': 41, 'suburban': 42, 'under': 43, 'grew': 44, 'failed': 45, 'detail': 46, 'half': 47, 'seen': 48, 'insensible': 49, 'minutes': 50, 'rest': 51, 'cigars': 52, 'strokes': 53, 'being': 54, 'pathos': 55, 'nothing': 56, 'rain': 57, 'tinge': 58, 'a': 59, 'way': 60, 'every': 61, 'over': 62, 'dimmest': 63, 'leading': 64, 'wife': 65, 'rs': 66, 'frame': 67, 'done': 68, 'attitude': 69, 'myself': 70, 'trade': 71, 'panelling': 72, 's

- Below are the first 50 entries in this vocabulary:

In [15]:
vocab_items = list(vocab.items())
i = 0
while i<50:
    print(vocab_items[i])
    i = i + 1


('', 0)
('cheeks', 1)
('romantic', 2)
('the', 3)
('come', 4)
('near', 5)
('often', 6)
('lean', 7)
('azaleas', 8)
('leathery', 9)
('mourn', 10)
('idle', 11)
('three', 12)
('most', 13)
('without', 14)
('just', 15)
('none', 16)
('reflection', 17)
('brings', 18)
('profusion', 19)
('fond', 20)
('equally', 21)
('fitting', 22)
('Only', 23)
('disdain', 24)
('wondered', 25)
('faith', 26)
('check', 27)
('Stroud', 28)
('patient', 29)
('underlay', 30)
('himself', 31)
('strolled', 32)
('pardonable', 33)
('might', 34)
('Lord', 35)
('reproduction', 36)
('seemed', 37)
('secret', 38)
('florid', 39)
('moment', 40)
('apparently', 41)
('suburban', 42)
('under', 43)
('grew', 44)
('failed', 45)
('detail', 46)
('half', 47)
('seen', 48)
('insensible', 49)


- Below, we illustrate the tokenization of a short sample text using a small vocabulary:

<img src="figures/05.png" width="800px">

- Let's now put it all together into a tokenizer class

In [20]:
class SimpleTokenizerV1:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {i:s for s,i in vocab.items()}
    
    def encode(self, text):
        preprocessed = re.split(r'([,.?_!"()\']|--|\s)', text)
        preprocessed = [
            item.strip() for item in preprocessed if item.strip()
        ]
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids
        
    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        # Replace spaces before the specified punctuations
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
        return text

- The `encode` function turns text into token IDs
- The `decode` function turns token IDs back into text

<img src="figures/06.png" width="800px">

- We can use the tokenizer to encode (that is, tokenize) texts into integers
- These integers can then be embedded (later) as input of/for the LLM

In [26]:
tokenizer = SimpleTokenizerV1(vocab)

text = "It's the last he painted, you know, Mrs. Gisburn said with pardonable pride."
ids = tokenizer.encode(text)
print(ids)

# print(SimpleTokenizerV1(vocab).encode(text)) another way to do the above task


[218, 603, 756, 3, 1081, 1044, 689, 737, 551, 868, 737, 342, 77, 1041, 369, 671, 33, 755, 77]


- We can decode the integers back into text

In [27]:
tokenizer.decode(ids)

"It' s the last he painted, you know, Mrs. Gisburn said with pardonable pride."

<br>
<br>
<br>
<br>

# 2.3 BytePair encoding

- GPT-2 used BytePair encoding (BPE) as its tokenizer
- it allows the model to break down words that aren't in its predefined vocabulary into smaller subword units or even individual characters, enabling it to handle out-of-vocabulary words
- For instance, if GPT-2's vocabulary doesn't have the word "unfamiliarword," it might tokenize it as ["unfam", "iliar", "word"] or some other subword breakdown, depending on its trained BPE merges
- The original BPE tokenizer can be found here: [https://github.com/openai/gpt-2/blob/master/src/encoder.py](https://github.com/openai/gpt-2/blob/master/src/encoder.py)
- In this lecture, we are using the BPE tokenizer from OpenAI's open-source [tiktoken](https://github.com/openai/tiktoken) library, which implements its core algorithms in Rust to improve computational performance
- (Based on an analysis [here](https://github.com/rasbt/LLMs-from-scratch/blob/main/ch02/02_bonus_bytepair-encoder/compare-bpe-tiktoken.ipynb), I found that `tiktoken` is approx. 3x faster than the original tokenizer and 6x faster than an equivalent tokenizer in Hugging Face)

In [ ]:
# pip install tiktoken

In [2]:
import importlib
import tiktoken

print("tiktoken version:", importlib.metadata.version("tiktoken"))

tiktoken version: 0.9.0


In [30]:
tokenizer = tiktoken.get_encoding("gpt2")

In [ ]:
text = (
    "Hello, do you like tea? <|endoftext|> In the sunlit terraces"
     "of someunknownPlace."
)

integers = tokenizer.encode(text, allowed_special={"<|endoftext|>"})
# The <|endoftext|> token is equal to 50256 which is the total no. of tokens in the GPT2 model BPE Tokenizer
print(integers)

[15496, 11, 466, 345, 588, 8887, 30, 220, 50256, 554, 262, 4252, 18250, 8812, 2114, 1659, 617, 34680, 27271, 13]


In [42]:
# This BPE Tokenizer can even hanle unknown words by converting them into subwords and then into tokens
print(tokenizer.encode("Kushagra")) 
print(tokenizer.decode([42]))
print(tokenizer.decode([1530]))
print(tokenizer.decode([45429]))

[42, 1530, 45429]
K
ush
agra


In [43]:
strings = tokenizer.decode(integers)

print(strings)

Hello, do you like tea? <|endoftext|> In the sunlit terracesof someunknownPlace.


- BPE tokenizers break down unknown words into subwords and individual characters:

<img src="figures/07.png" width="700px">

In [44]:
tokenizer.encode("Akwirw ier", allowed_special={"<|endoftext|>"})

[33901, 86, 343, 86, 220, 959]

<br>
<br>
<br>
<br>

# 2.4 Data sampling with a sliding window

- Above, we took care of the tokenization (converting text into word tokens represented as token ID numbers)
- Now, let's talk about how we create the data loading for LLMs
- We train LLMs to generate one word at a time, so we want to prepare the training data accordingly where the next word in a sequence represents the target to predict

<img src="figures/08.png" width="800px">

- For this, we use a sliding window approach, changing the position by +1:

<img src="figures/09.png" width="900px">

- Note that in practice it's best to set the stride equal to the context length so that we don't have overlaps between the inputs (the targets are still shifted by +1 always)

<img src="figures/10.png" width="800px">

In [ ]:
# creating a dataloader
from supplementary import create_dataloader_v1

# raw_text is the text that we read from the file "the-verdict.txt"
# max_length is the maximum column length of the input tensor
# stride is the number of positions to move the window by
# batch_size is the number of rows in each batch (i.e here there are 8 rows in each batch)
# shuffle is used to shuffle the data before creating the batches (here we have made shuffle as False)
dataloader = create_dataloader_v1(raw_text, batch_size=8, max_length=4, stride=4, shuffle=False)

data_iter = iter(dataloader)
inputs, targets = next(data_iter)
# we have now prepared our data set for training of the LLM
print("Inputs:\n", inputs)
print("\nTargets:\n", targets) # The targets are the same as the inputs but shifted by one position

Inputs:
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])

Targets:
 tensor([[  367,  2885,  1464,  1807],
        [ 3619,   402,   271, 10899],
        [ 2138,   257,  7026, 15632],
        [  438,  2016,   257,   922],
        [ 5891,  1576,   438,   568],
        [  340,   373,   645,  1049],
        [ 5975,   284,   502,   284],
        [ 3285,   326,    11,   287]])


<br>
<br>
<br>
<br>

# Exercise: Prepare your own favorite text dataset